In [11]:
#!pip show torch
#!pip show torchtext

In [20]:
#!pip install torch torchvision scikit-learn
#!pip install --upgrade torch torchtext
#!pip install nltk
#!pip install torch==2.4.1 torchtext==0.18.0
#!pip install --upgrade torch torchtext
# !pip uninstall torch torchtext
#!pip install torchtext==0.18.1
!pip install --upgrade torchtext



In [1]:
# Import necessary libraries
import pandas as pd
import torch
import nltk
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("data/movies_2015_2023_preprocessed_genre.csv")
df.head()

,title,release_year,language,genre,overview,vote_average,vote_count,popularity,cleaned_overview
0,Pad Man,2018,Indisch,Humor,upon realizing extent woman affected menses se...,7.420,200.0,7.036,upon realizing extent woman affected menses se...
1,Tamasha,2015,Indisch,Humor,meeting vacation ved tara sense connection vow...,6.720,141.0,8.770,meeting vacation ved tara sense connection vow...
2,Tu Jhoothi Main Makkaar,2023,Indisch,Humor,earn extra cash mickey help couple break life ...,6.253,144.0,10.045,earn extra cash mickey help couple break life ...
3,Hindi Medium,2017,Indisch,Humor,mita raj batra affluent couple delhi chandni c...,7.300,166.0,7.001,mita raj batra affluent couple delhi chandni c...
4,Dilwale,2015,Indisch,Humor,raj mafia member one day meet girl meera chasi...,6.648,301.0,11.501,raj mafia member one day meet girl meera chasi...


In [3]:
# Fill NaN values in 'cleaned_overview' with an empty string
df['cleaned_overview'] = df['cleaned_overview'].fillna('')

In [4]:
# Drop rows with missing cleaned_overview
df = df.dropna(subset=['cleaned_overview'])

In [6]:
from sklearn.preprocessing import LabelEncoder
# Convert the 'language' column to numeric labels using LabelEncoder
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['language'])

In [7]:
# Create value tuples (label, cleaned_overview)
value_tuples = []
for _, row in df.iterrows():
    value_tuples.append((row['label'], row['cleaned_overview']))

# Print the length and a few examples
print(f"Total number of value tuples: {len(value_tuples)}")
print("First tuple:", value_tuples[0])
print("Last tuple:", value_tuples[-1])

Total number of value tuples: 15500
First tuple: (3, 'upon realizing extent woman affected menses set create sanitary pad machine provide inexpensive sanitary pad woman rural india')
Last tuple: (4, 'several european underground director turn ancient rule mankind inside shocking goresoaked interpretation god law bible world ending commandment dying')


In [8]:
from sklearn.model_selection import train_test_split

# Split the data into 80% training and 20% testing
train_value_tuples, test_value_tuples = train_test_split(value_tuples, test_size=0.2, random_state=42)

# Print the size of the training and testing sets
print(f"Training set size: {len(train_value_tuples)}")
print(f"Testing set size: {len(test_value_tuples)}")


Training set size: 12400
Testing set size: 3100


## Preparing data processing pipelines

In [21]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

train_iter = iter(train_value_tuples)

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

OSError: /usr/local/python/3.12.1/lib/python3.12/site-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs

In [22]:
# # Named entities, stop words and punctuation are STILL part of the vocabulary

example_text = 'So obsessed is she with cinema, that she worships the popular actor Jaishankar'
print('raw:',example_text)
print('cleaned:',tokenizer(example_text))
print('bow:',vocab(tokenizer(example_text)))
print('\nvocab length:', len(vocab))

raw: So obsessed is she with cinema, that she worships the popular actor Jaishankar


NameError: name 'tokenizer' is not defined

In [ ]:
text_pipeline = lambda x: vocab(tokenizer(x))

# label pipeline does not have to do anything since values are already 0 or 1
label_pipeline = lambda x: int(x)

In [ ]:
text_pipeline('here is an natarajan')

In [ ]:
label_pipeline(0)

## Generate data batch and iterator

In [23]:
from torch.utils.data import DataLoader
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
         text_list.append(processed_text)
         offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

# 2: Vectorization

#### Bag-of-Words

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the CountVectorizer (Bag-of-Words)
bow_vectorizer = CountVectorizer(max_features=5000)

# Fit and transform the training data
X_train_bow = bow_vectorizer.fit_transform(X_train)

# Transform the test data
X_test_bow = bow_vectorizer.transform(X_test)


#### TF-IDF

In [32]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# PyTorch Classification Model

In [37]:
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

class TextClassificationModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TextClassificationModel, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        return self.fc(x)

# Define the model, loss, and optimizer
input_dim = X_train_bow.shape[1]  # Adjust according to your input size (5000 for Bag-of-Words/TF-IDF)
num_classes = len(y.unique())  # Number of unique languages
model = TextClassificationModel(input_dim=input_dim, num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (simplified)
def train_model(model, X_train, y_train):
    model.train()
    for epoch in range(5):  # Training for 5 epochs
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch+1}/5], Loss: {loss.item():.4f}')

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

# Convert Bag-of-Words to torch tensor and train
X_train_tensor = torch.tensor(X_train_bow.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

train_model(model, X_train_tensor, y_train_tensor)


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [38]:
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder

class TextClassificationModel(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(TextClassificationModel, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
    
    def forward(self, x):
        return self.fc(x)

# Define the model, loss, and optimizer
input_dim = X_train_bow.shape[1]  # Adjust according to your input size (5000 for Bag-of-Words/TF-IDF)
num_classes = len(y.unique())  # Number of unique languages
model = TextClassificationModel(input_dim=input_dim, num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (simplified)
def train_model(model, X_train, y_train):
    model.train()
    for epoch in range(5):  # Training for 5 epochs
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        print(f'Epoch [{epoch+1}/5], Loss: {loss.item():.4f}')

# Convert TF-IDF to torch tensor and train
X_train_tensor = torch.tensor(X_train_tfidf.toarray(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

train_model(model, X_train_tensor, y_train_tensor)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [39]:
from sklearn.metrics import accuracy_score, confusion_matrix

# Get model predictions
model.eval()
with torch.no_grad():
    y_pred = model(X_test_tensor).argmax(dim=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_tensor, y_pred)
print(f'Accuracy: {accuracy:.4f}')

# Confusion matrix
conf_matrix = confusion_matrix(y_test_tensor, y_pred)
print(conf_matrix)


NameError: name 'X_test_tensor' is not defined